In [1]:
import json
import re
import pandas as pd
import random
from dotenv import load_dotenv
from llama_index.core import (
    Document,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
    get_response_synthesizer,
)
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.evaluation import RetrieverEvaluator
from llama_index.core.prompts import PromptTemplate
from llama_index.core.schema import QueryBundle
from llama_index.core.settings import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

In [2]:
# Carga del texto limpio
data_path = "../data/plain_text/plain_text.txt"
with open(data_path, "r", encoding = "utf-8") as f:
    content = f.read()

# Generar el document
pdf_doc = Document(text=content.strip())

# Carga de las preguntas
qa_path = "../data/exams/clean_questions/questions_all_tests.json"
with open(qa_path, "r", encoding="utf-8") as f:
    questions = json.load(f)

# Export boolean
export_csv = False

# Key
load_dotenv()
# Set embedding model
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

# Primera Prueba

In [ ]:
# LLM Model
system_prompt = """
Eres un experto en Apache Spark y PySpark, y estás respondiendo preguntas tipo test usando exclusivamente la información del contexto proporcionado.

No puedes utilizar ningún conocimiento previo ni información externa al contexto.

Tu tarea es:
1. Leer el contexto cuidadosamente.
2. Leer la pregunta y las opciones tipo test.
3. Seleccionar la opción correcta **solo si** está claramente justificada por el contenido del contexto.
4. Justificar tu elección **citando explícitamente fragmentos relevantes del contexto**.

Si no puedes justificar una respuesta a partir del contexto, responde exactamente "No he encontrado información al respecto." y no devuelvas ninguna opción como respuesta.
"""
llm = OpenAI(
    model="gpt-4o-mini",
    temperature=0,
    system_prompt=system_prompt
)

Settings.llm = llm
# Set embedding model
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

In [3]:
# Ruta al índice optimizado
storage_path = "../data/index_storage/"
storage_context = StorageContext.from_defaults(persist_dir=storage_path)
index = load_index_from_storage(storage_context)

# Configurar el retriever
retriever = index.as_retriever(similarity_top_k=5)

In [4]:
# Función auxiliar para formatear las opciones:
def format_options(options_dict):
    clean_options = {}
    for key in options_dict:
        # Elimina letras errantes como '\nB' o '\nC' dentro del texto
        value = options_dict[key]
        value = re.sub(r'\n[A-E]\n?', '', value).strip()
        clean_options[key] = value
    return "\n".join([f"{k}. {v}" for k, v in clean_options.items()])

# Función para lanzar la query:
def ask_question_with_options(question_obj, 
                              #query_str_template, 
                              retriever = retriever, response_mode= "compact"):
    question_text = question_obj["question"]
    options_text = format_options(question_obj["options"])

    #qa_prompt = PromptTemplate(query_str_template)
    #Settings.text_qa_template = qa_prompt

    query_str = f"""
    {question_text}

    Opciones: 
    {options_text}
        
    IMPORTANTE:
        - Usa solo información del contexto.
        - No respondas con conocimientos externos.
        - Si no puedes justificar la respuesta con fragmentos del contexto, no devuelvas ninguna opción.
        - Si puedes justificar la respuesta con fragmentos del contexto, justificala
    """
    
    # Usar QueryBundle para pasar la información
    query_bundle = QueryBundle(
        query_str = query_str,
        custom_embedding_strs = [question_text, options_text]
    )

    response_synthesizer = get_response_synthesizer(response_mode=response_mode)
    
    query_engine = RetrieverQueryEngine(
        retriever=retriever,
        response_synthesizer=response_synthesizer
    )
    
    response = query_engine.query(query_bundle)
    predicted_answer = response.response.strip()

    return {
        "question_id": question_obj["question_id"],
        "question" : question_obj["question"],
        "options": format_options(question_obj["options"]),
        "predicted_answer": predicted_answer,
        "correct_answer": question_obj["correct_answer"],
        "is_correct": predicted_answer == question_obj["correct_answer"],
        "response_obj": response  # Para inspeccionar source_nodes si se desea
    }

# Validar la repsuesta:
def display_question_result(result):
    print("ID Pregunta:", result["question_id"])
    print("\nPregunta:\n", result["question"])
    print("\nOpciones:\n", result["options"])
    print("\nAcertó?:", result["is_correct"])
    print("\nRespuesta generada por el modelo:\n", result["response_obj"])
    print("\nRespuesta correcta:", result["correct_answer"])
    if "response_obj" in result:
        print(f"\n🔎 Chunks usados para responder la pregunta {result['question_id']}:\n")
        for i, node in enumerate(result["response_obj"].source_nodes):
            print(f"--- Chunk #{i+1} | Score: {node.score:.4f} ---")
            print(node.node.get_content().strip())
            print()

In [ ]:
question =[q for q in questions if q["question_id"] == "Test_2_39"][0]
test_result_2_39 = ask_question_with_options(question, response_mode = "compact")
# Validation
display_question_result(test_result_2_39)

In [ ]:
question =[q for q in questions if q["question_id"] == "Test_1_27"][0]
test_result_2_39 = ask_question_with_options(question, response_mode = "compact")
# Validation
display_question_result(test_result_2_39)

In [ ]:
question =[q for q in questions if q["question_id"] == "Test_2_54"][0]
test_result_2_39 = ask_question_with_options(question, response_mode = "compact")
# Validation
display_question_result(test_result_2_39)

In [ ]:
# Test_2_18, Test_1_9, Test_2_12, Test_1_53, Test_1_41

ids = ["Test_2_18", "Test_1_9", "Test_2_12", "Test_1_53", "Test_1_41"]

for id in ids: 
    question =[q for q in questions if q["question_id"] == id][0]
    #result_random = ask_question_with_options(question, response_mode = "refine")
    result_random = ask_question_with_options(question, response_mode = "tree_summarize")
    display_question_result(result_random)
    print("="*20)

In [ ]:
ids = ["Test_1_53", "Test_1_17", "Test_2_28", "Test_2_31", "Test_2_29"]

for id in ids: 
    question =[q for q in questions if q["question_id"] == id][0]
    #result_random = ask_question_with_options(question, response_mode = "refine")
    result_random = ask_question_with_options(question, response_mode = "tree_summarize")
    display_question_result(result_random)
    print("="*20)

In [ ]:
random_questions = random.sample(questions, k=5)
for q in random_questions: 
    result_random = ask_question_with_options(q, response_mode = "refine")
    # result_random = ask_question_with_options(q, response_mode = "tree_summarize")
    display_question_result(result_random)
    print("="*20)

In [ ]:
result_random = ask_question_with_options(random.choice(questions), response_mode = "refine")
display_question_result(result_random)

# System Prompt menos restrictivo - Segunda Prueba

* Este prompt permitiría razonar desde el contenido si no está claramente la respuesta en el contexto.

In [ ]:
system_prompt = """
Eres un experto en Apache Spark y PySpark, y estás respondiendo preguntas tipo test usando exclusivamente la información del contexto proporcionado.

No puedes utilizar ningún conocimiento previo ni información externa al contexto.

Tu tarea es:
1. Leer el contexto cuidadosamente.
2. Leer la pregunta y las opciones tipo test.
3. Seleccionar la opción correcta si está claramente justificada por el contenido del contexto, y justificar tu elección **citando explícitamente fragmentos relevantes del contexto**.
4. En caso de no estar claramente justificada por el contexto, intentar razonar desde el contenido del contexto una respuesta, indicando qué información se ha utilizado, pero NO UTILICES INFORMACIÓN EXTERNA
5. En caso de no encontrar información en el contexto relacionada con la pregunta, devolver "No existe información" aunque sepas la respuesta correcta.
"""

* Permite razonar solo si se citan y conectan fragmentos de forma explícita.

In [26]:
system_prompt = """
Eres un experto en Apache Spark y PySpark, y estás respondiendo preguntas tipo test usando exclusivamente la información del contexto proporcionado.

No puedes utilizar ningún conocimiento previo ni información externa al contexto.

Tu tarea es:
1. Leer el contexto cuidadosamente.
2. Leer la pregunta y las opciones tipo test.
3. Seleccionar la opción correcta si está claramente justificada por el contenido del contexto, y justificar tu elección citando fragmentos textuales exactos.
4. Si no hay una justificación directa, puedes razonar una respuesta **únicamente si se basa en información presente en el contexto**, indicando exactamente **qué fragmentos usas** y **cómo los conectas**. No está permitido inventar datos.
5. En caso de no encontrar ninguna información útil en el contexto, responde exactamente con: **"No existe información"**
"""

In [27]:
llm = OpenAI(
    model="gpt-4o-mini",
    temperature=0,
    system_prompt=system_prompt
)

Settings.llm = llm

In [28]:
def extract_predicted_letter(response_text):
    """
    Extrae la letra (A, B, C, D o E) de una respuesta generada en el formato:
    'Respuesta correcta: <LETRA>\nJustificación: <...>'
    """
    match = re.search(r"Respuesta\s+correcta\s*:\s*([A-E])", response_text, flags=re.IGNORECASE)
    if match:
        return match.group(1).upper()
    return None 

def ask_question_with_options(question_obj, 
                              #query_str_template, 
                              retriever = retriever, response_mode= "compact"):
    question_text = question_obj["question"]
    options_text = format_options(question_obj["options"])

    #qa_prompt = PromptTemplate(query_str_template)
    #Settings.text_qa_template = qa_prompt

    query_str = f"""
    {question_text}

    Opciones: 
    {options_text}
        
    IMPORTANTE:
        - Usa solo información del contexto.
        - No respondas con conocimientos externos.
        - Aunque sepas la respuesta correcta, si no existe esa información en el contexto devuelve "No existe información"
    
    Responde de la siguiente manera:
    "
    Respuesta correcta: <LETRA>
    Justificación: <JUSTIFICACIÓN>
    "
    """
    
    # Usar QueryBundle para pasar la información
    query_bundle = QueryBundle(
        query_str = query_str,
        custom_embedding_strs = [question_text, options_text]
    )

    response_synthesizer = get_response_synthesizer(response_mode=response_mode)
    
    query_engine = RetrieverQueryEngine(
        retriever=retriever,
        response_synthesizer=response_synthesizer
    )
    
    response = query_engine.query(query_bundle)
    predicted_answer = response.response.strip()
    
    return {
        "question_id": question_obj["question_id"],
        "question" : question_obj["question"],
        "options": format_options(question_obj["options"]),
        "predicted_answer": predicted_answer,
        "correct_answer": question_obj["correct_answer"],
        "is_correct": extract_predicted_letter(predicted_answer) == question_obj["correct_answer"],
        "response_obj": response
    }


In [ ]:
# Preguntas que no razonaba:
ids = ["Test_1_17", "Test_2_28", "Test_2_31", "Test_1_41"]
for id in ids: 
    question =[q for q in questions if q["question_id"] == id][0]
    # result_random = ask_question_with_options(question, response_mode = "refine") # Sirve para Test_1_41 pero no para el resto. 
    result_random = ask_question_with_options(question, response_mode = "tree_summarize")
    display_question_result(result_random)
    print("="*20)

In [24]:
random_questions = random.sample(questions, k=3)

In [25]:
for question in random_questions: 
    
    print("Response mode: TREE SUMMARIZE")
    print("="*20)
    result_random = ask_question_with_options(question, response_mode = "tree_summarize")
    display_question_result(result_random)
    print("="*20)

Response mode: TREE SUMMARIZE
ID Pregunta: Test_1_21

Pregunta:
 Which of the following code blocks performs an inner join of the salarydf and employeedf 
DataFrames for columns employeeSalaryID and employeeID, respectively?

Opciones:
 A. salarydf.join(employeedf, salarydf.employeeID == employeedf.
employeeSalaryID)
B. i.	
Salarydf.createOrReplaceTempView(salarydf)
ii.	 employeedf.createOrReplaceTempView('employeedf')
iii.	 spark.sql("SELECT * FROM salarydf CROSS JOIN employeedf ON 
employeeSalaryID ==employeeID")
C. i.	
salarydf
ii.	 .join(employeedf, col(employeeID)==col(employeeSalaryID))
D. i.	
Salarydf.createOrReplaceTempView(salarydf)
ii.	 employeedf.createOrReplaceTempView('employeedf')
iii.	 SELECT * FROM salarydf
iv.	 INNER JOIN employeedf
v.	
ON salarydf.employeeSalaryID == employeedf. employeeID

Acertó?: False

Respuesta generada por el modelo:
 No existe información

Respuesta correcta: D

🔎 Chunks usados para responder la pregunta Test_1_21:

--- Chunk #1 | Score: 0.7563

In [ ]:
    print("="*20)
    print("Response mode: REFINE")
    print("="*20)
    result_random = ask_question_with_options(question, response_mode = "refine") # Sirve para Test_1_41 pero no para el resto. 
    display_question_result(result_random)
    print("="*20)

In [18]:
# Test_2_57
question =[q for q in questions if q["question_id"] == "Test_2_57"][0]
test_result = ask_question_with_options(question, response_mode = "tree_summarize")
# Validation
display_question_result(test_result)

ID Pregunta: Test_2_57

Pregunta:
 Which code block will write the df DataFrame as a parquet file on the filePath path partitioning 
it on the department column?

Opciones:
 A. df.write.partitionBy("department").parquet(filePath)
B. df.write.partition("department").parquet(filePath)
C. df.write.parquet("department").partition(filePath)
D. df.write.coalesce("department").parquet(filePath)

Acertó?: False

Respuesta generada por el modelo:
 No existe información

Respuesta correcta: A

🔎 Chunks usados para responder la pregunta Test_2_57:

--- Chunk #1 | Score: 0.5939 ---
Advanced Operations and Optimizations in Spark

Reading and writing Parquet files
In this section, we will discuss the Parquet file format. Parquet is a columnar file format that makes data 
reading and writing very efficient. It is also a compact file format that facilitates faster reads and writes.
Let’s learn how to write Parquet files with Spark by running the following code:
salary_data_with_id.write.parquet('salar

In [29]:
# Test_1_12 
question =[q for q in questions if q["question_id"] == "Test_1_12"][0]
test_result = ask_question_with_options(question, response_mode = "tree_summarize")
# Validation
display_question_result(test_result)

ID Pregunta: Test_1_12

Pregunta:
 Which of the following is one of the tasks of the cluster manager in Spark?

Opciones:
 A. In the event of an executor failure, the cluster manager will collaborate with the driver to 
initiate a new executor
B. The cluster manager can coalesce partitions to increase the speed of complex data processing
C. The cluster manager collects runtime statistics of queries
D. The cluster manager creates query plans

Acertó?: True

Respuesta generada por el modelo:
 Respuesta correcta: A  
Justificación: "The cluster manager is responsible for launching the driver program and allocating resources for execution." Además, se menciona que "the driver has all the information about the number of available workers and the tasks that are running on each of them alongside data in case a worker fails, that task can be reassigned to a different cluster." Esto implica que el cluster manager colabora con el driver en la gestión de recursos y la re-asignación de tareas en c